In [1]:
import requests
from bs4 import BeautifulSoup
import re
from pprint import pprint

In [2]:
result = requests.get('https://comic.naver.com/webtoon/weekday.nhn')

soup = BeautifulSoup(result.content, 'lxml')
soup

<!DOCTYPE html>
<html lang="ko">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<title>네이버 만화 &gt; 요일별  웹툰 &gt; 전체웹툰</title>
<meta content="네이버 웹툰" property="og:title"/>
<meta content="https://ssl.pstatic.net/static/comic/images/og_tag_v2.png" property="og:image"/>
<meta content="매일매일 새로운 재미, 네이버 웹툰." property="og:description"/>
<meta content="https://comic.naver.com/webtoon/weekday.nhn" property="og:url"/>
<meta content="article" property="og:type"/>
<meta content="네이버 웹툰" property="og:article:author"/>
<meta content="https://comic.naver.com" property="og:article:author:url"/>
<link href="https://ssl.pstatic.net/static/comic/favicon/webtoon_favicon_32x32.ico" rel="shortcut icon" type="image/x-icon"/>
<script type="text/javascript">
		document.domain = "naver.com";
	</script>
<link href="/css/comic/common_20190605124420.css" rel="stylesheet" style="text/css"/>
<link href="/css/comic/comic_20190

In [3]:
week_cols = soup.select('div.col div.col_inner')

webtoon_dict = {}
for col in week_cols:
    weekday = col.find('span')
#     print(weekday.text)
    webtoons = col.select('ul li a')

    for webtoon in webtoons:
        title = webtoon.text
        link = webtoon['href']
        titleId = re.findall('[0-9]+',link)
#         print(titleId)
        if not '\n' in title:
            webtoon_dict[title]  = {
                'title': title,
                'link': link,
                'title_id': titleId[0]
            }
pprint(webtoon_dict)

{'12차원 소년들': {'link': '/webtoon/list.nhn?titleId=717059&weekday=wed',
              'title': '12차원 소년들',
              'title_id': '717059'},
 '1초': {'link': '/webtoon/list.nhn?titleId=725586&weekday=fri',
        'title': '1초',
        'title_id': '725586'},
 '203호 저승사자': {'link': '/webtoon/list.nhn?titleId=670140&weekday=wed',
               'title': '203호 저승사자',
               'title_id': '670140'},
 '27-10': {'link': '/webtoon/list.nhn?titleId=725217&weekday=sun',
           'title': '27-10',
           'title_id': '725217'},
 '2인용 인간': {'link': '/webtoon/list.nhn?titleId=710769&weekday=sat',
            'title': '2인용 인간',
            'title_id': '710769'},
 '5kg을 위하여': {'link': '/webtoon/list.nhn?titleId=710639&weekday=sat',
              'title': '5kg을 위하여',
              'title_id': '710639'},
 '8월의 눈보라': {'link': '/webtoon/list.nhn?titleId=703856&weekday=sun',
             'title': '8월의 눈보라',
             'title_id': '703856'},
 'Doll 체인지': {'link': '/webtoon/list.nhn?titleId=6

- href 는 a 태그의 하이퍼링크 url이 들어가 있다.
- 그런데!! href 값을 보니 /부터 시작한다?
- /~~ = 리소스경로!! (+ ?~~ = 쿼리스트링)

**결론**
- '프로토콜://주소값/' 가져와서 이거 뒤에 href값(리소스경로+쿼리스트링) 붙이자.
- 어떤 프로토콜://주소값을 긁어와야되는지는 어떻게 알지? = href 클릭해보면 안다.

In [4]:
url_webtoon = 'http://comic.naver.com'

for idx, toon in enumerate(webtoon_dict):
    result = requests.get(url_webtoon + webtoon_dict[toon]['link'])
    soup = BeautifulSoup(result.content)
    
    current_href = soup.select('table.viewList tr td.title a')[0]['href']
    current_no = re.findall('no=([0-9]+)', current_href)[0]
    print(current_no)
    webtoon_dict[toon]['current_no'] = current_no
    if (idx+1)%40 == 0:
        print('{} 완료.. {}/{}'.format(toon, idx+1, len(webtoon_dict)))

433
219
262
199
260
48
85
56
813
6
148
47
6
99
45
98
61
98
24
50
671
119
2
4
56
16
11
12
11
65
134
93
82
79
19
15
31
24
63
1178
마음의소리 완료.. 40/239
82
1331
117
99
246
65
251
113
299
151
66
331
170
112
535
155
139
59
57
8
21
29
12
24
44
11
30
52
50
27
492
44
128
21
255
390
184
177
59
13
아도나이 완료.. 80/239
55
71
5
161
87
5
35
179
2
12
31
79
37
96
251
84
22
129
89
62
188
213
74
30
39
105
53
41
32
264
282
42
301
53
27
97
5
404
91
177
가담항설 완료.. 120/239
7
3
86
27
34
229
8
57
17
118
11
57
107
115
329
40
58
169
2
6
8
55
38
48
1
238
418
87
8
280
208
162
13
53
58
19
89
64
17
55
하르모니아 완료.. 160/239
220
34
19
8
5
137
18
34
49
20
51
61
9
1
1
160
212
289
150
219
58
29
8
16
23
472
79
8
43
91
12
53
66
56
41
12
2
110
3
40
고인의 명복 완료.. 200/239
3
15
41
23
1
177
104
58
15
214
292
192
26
101
5
28
127
106
107
25
596
4
105
8
26
16
3
11
14
49
30
102
71
1
32
19
2
18
51


In [5]:
pprint(webtoon_dict)

{'12차원 소년들': {'current_no': '39',
              'link': '/webtoon/list.nhn?titleId=717059&weekday=wed',
              'title': '12차원 소년들',
              'title_id': '717059'},
 '1초': {'current_no': '13',
        'link': '/webtoon/list.nhn?titleId=725586&weekday=fri',
        'title': '1초',
        'title_id': '725586'},
 '203호 저승사자': {'current_no': '84',
               'link': '/webtoon/list.nhn?titleId=670140&weekday=wed',
               'title': '203호 저승사자',
               'title_id': '670140'},
 '27-10': {'current_no': '14',
           'link': '/webtoon/list.nhn?titleId=725217&weekday=sun',
           'title': '27-10',
           'title_id': '725217'},
 '2인용 인간': {'current_no': '56',
            'link': '/webtoon/list.nhn?titleId=710769&weekday=sat',
            'title': '2인용 인간',
            'title_id': '710769'},
 '5kg을 위하여': {'current_no': '118',
              'link': '/webtoon/list.nhn?titleId=710639&weekday=sat',
              'title': '5kg을 위하여',
              'title_id': '710

여기서부터 내 코드

- requests.get(url, params)를 했을 때 result.content에 실패 한다면
1. network에 들어가서 요청헤더(requests header)를 확인하자.
2. 명시되어 있는 애들을 복사해서 headers에 dictionary형태로 넣자
3. requests.get(url,params, headers) 추가해서 요청하자

In [6]:
comment_url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?'
params = {'ticket':'comic',
         'templateId' : 'webtoon',
         'pool' : 'cbox3',
         '_callback' : 'jQuery112407869252129422144_1560319769583',
         'lang' : 'ko',
         'country' : 'KR',
         'objectId' : '626907_255', #title_cuurentid no.    -> 여기에 format줘서 각 웹툰의 id와 current no.넣을거
         'pageSize' : 15,      # 댓글의 갯수를 말하는듯
         'indexSize' : 10,
         'listType' :'OBJECT',
         'pageType' : 'default',
         'page' : 1,         #page 바꿔가면서 댓글 뽑아보자
         'refresh' : 'true',
         'sort' : 'new',
         '_' : 1560319769587}
#요청 헤더를 추가해보자
headers = {
    'Referer' : 'https://comic.naver.com/comment/comment.nhn?titleId=626907&no=255'  #  titleid와 current no.가 웹툼마다 달라 이걸 알려주면 어떤 웹툰에 들어갈지 파악하고 들어간다.
}                                                                                  # 전체 웹툰의 댓글 보기 위해선 웹툰의 각 화를 들어가서 읽어야한다. 그래서 위에서 titleid를 읽은것

#result = requests.get(comment_url,params = params)
result = requests.get(comment_url, headers=headers ,params = params)

result

<Response [200]>

In [7]:
result.content

b'jQuery112407869252129422144_1560319769583({"success":true,"code":"1000","message":"\xec\x9a\x94\xec\xb2\xad\xec\x9d\x84 \xec\x84\xb1\xea\xb3\xb5\xec\xa0\x81\xec\x9c\xbc\xeb\xa1\x9c \xec\xb2\x98\xeb\xa6\xac\xed\x95\x98\xec\x98\x80\xec\x8a\xb5\xeb\x8b\x88\xeb\x8b\xa4.","lang":"ko","country":"KR","result":{"commentList":[{"ticket":"comic","objectId":"626907_255","categoryId":"*","templateId":"default","commentNo":372022974,"parentCommentNo":372022974,"replyLevel":1,"replyCount":0,"replyAllCount":0,"replyPreviewNo":null,"replyList":null,"imageCount":0,"imageList":null,"imagePathList":null,"imageWidthList":null,"imageHeightList":null,"commentType":"txt","stickerId":null,"sticker":null,"sortValue":1560325629894,"contents":"\xec\x95\x84 \xe3\x85\x8b\xe3\x85\x8b\xe3\x85\x8b\xe3\x85\x8b\xe3\x85\x8b\xe3\x85\x8b\xe3\x85\x8b\xec\xa7\x84\xec\xa7\x9c \xe3\x85\x8b\xe3\x85\x8b\xe3\x85\x8b\xe3\x85\x8b\xe3\x85\x8b\xe3\x85\x8b\xec\x9b\x83\xea\xb2\xa8\xec\xa3\xbd\xea\xb2\xa0\xeb\x84\xa4 \xe3\x85\x8b\xe3

jQuery 이 적혀있는 것을 보니 json 파일!
html -> 교류하고 싶은 정보를 구조화 시킨것
    -> 태그를 지칭해서 원하는 정보 얻는다.
    -> 단점 : 구조가 복잡, 데이터의 길이가 길다
    -> 단점을 보완하기 위해 등장한 것이 json = 자바스크립트오브젝트노테이션
json -> html보다 간단
     -> key를 지칭해서 원하는 정보를 얻는다.

In [8]:
import json

In [9]:
#result.content(JSON)  ->  dict로 변환
result_json = json.loads(result.content)
result_json

# 이렇게 하니깐 안된다 -> {} 앞뒤로 쓸데없는 데이터가 있다 -> 없애버리자

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [10]:
import re
result_update = re.findall('769583\((.+)\);', result.text)[0]   #특수문자 인식하려면 \써줘야한다.
result_update

'{"success":true,"code":"1000","message":"요청을 성공적으로 처리하였습니다.","lang":"ko","country":"KR","result":{"commentList":[{"ticket":"comic","objectId":"626907_255","categoryId":"*","templateId":"default","commentNo":372022974,"parentCommentNo":372022974,"replyLevel":1,"replyCount":0,"replyAllCount":0,"replyPreviewNo":null,"replyList":null,"imageCount":0,"imageList":null,"imagePathList":null,"imageWidthList":null,"imageHeightList":null,"commentType":"txt","stickerId":null,"sticker":null,"sortValue":1560325629894,"contents":"아 ㅋㅋㅋㅋㅋㅋㅋ진짜 ㅋㅋㅋㅋㅋㅋ웃겨죽겠네 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ","userIdNo":"24aCX","exposedUserIp":null,"lang":"ko","country":"KR","idType":"naver","idProvider":"naver","userName":"uk222","userProfileImage":"https://profile-phinf.pstatic.net/26438/fd9e539b643899bc402a361eb0399300f28f70f2aaa57dc818e899753e5c751b.jpg","profileType":"naver","modTime":"2019-06-12T16:47:09+0900","modTimeGmt":"2019-06-12T07:47:09+0000","regTime":"2019-06-12T16:47:09+0900","regTimeGmt":"2019-06-12T07:47:09+0000","sympathyCou

In [11]:
result_json = json.loads(result_update)
result_json

{'success': True,
 'code': '1000',
 'message': '요청을 성공적으로 처리하였습니다.',
 'lang': 'ko',
 'country': 'KR',
 'result': {'commentList': [{'ticket': 'comic',
    'objectId': '626907_255',
    'categoryId': '*',
    'templateId': 'default',
    'commentNo': 372022974,
    'parentCommentNo': 372022974,
    'replyLevel': 1,
    'replyCount': 0,
    'replyAllCount': 0,
    'replyPreviewNo': None,
    'replyList': None,
    'imageCount': 0,
    'imageList': None,
    'imagePathList': None,
    'imageWidthList': None,
    'imageHeightList': None,
    'commentType': 'txt',
    'stickerId': None,
    'sticker': None,
    'sortValue': 1560325629894,
    'contents': '아 ㅋㅋㅋㅋㅋㅋㅋ진짜 ㅋㅋㅋㅋㅋㅋ웃겨죽겠네 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
    'userIdNo': '24aCX',
    'exposedUserIp': None,
    'lang': 'ko',
    'country': 'KR',
    'idType': 'naver',
    'idProvider': 'naver',
    'userName': 'uk222',
    'userProfileImage': 'https://profile-phinf.pstatic.net/26438/fd9e539b643899bc402a361eb0399300f28f70f2aaa57dc818e899753e5c751b.jpg',
  

In [12]:
result_json['result']['commentList']

[{'ticket': 'comic',
  'objectId': '626907_255',
  'categoryId': '*',
  'templateId': 'default',
  'commentNo': 372022974,
  'parentCommentNo': 372022974,
  'replyLevel': 1,
  'replyCount': 0,
  'replyAllCount': 0,
  'replyPreviewNo': None,
  'replyList': None,
  'imageCount': 0,
  'imageList': None,
  'imagePathList': None,
  'imageWidthList': None,
  'imageHeightList': None,
  'commentType': 'txt',
  'stickerId': None,
  'sticker': None,
  'sortValue': 1560325629894,
  'contents': '아 ㅋㅋㅋㅋㅋㅋㅋ진짜 ㅋㅋㅋㅋㅋㅋ웃겨죽겠네 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
  'userIdNo': '24aCX',
  'exposedUserIp': None,
  'lang': 'ko',
  'country': 'KR',
  'idType': 'naver',
  'idProvider': 'naver',
  'userName': 'uk222',
  'userProfileImage': 'https://profile-phinf.pstatic.net/26438/fd9e539b643899bc402a361eb0399300f28f70f2aaa57dc818e899753e5c751b.jpg',
  'profileType': 'naver',
  'modTime': '2019-06-12T16:47:09+0900',
  'modTimeGmt': '2019-06-12T07:47:09+0000',
  'regTime': '2019-06-12T16:47:09+0900',
  'regTimeGmt': '2019-06-12T07:47:0

In [13]:
commentList = result_json['result']['commentList']
for i, comment in enumerate(commentList): #comment = dictionary
    print(i+1,comment['contents'])

1 아 ㅋㅋㅋㅋㅋㅋㅋ진짜 ㅋㅋㅋㅋㅋㅋ웃겨죽겠네 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
2 이래서 민중은 개돼지라는 말이나오는구나...
3 우기명 머리뿐만아니아 얼굴도 변한거같은데
4 진짜 오늘 존나웃기긴햇다 간만에 ㅋㅋㅋㅋㅋㅌ
5 아니 이게 어째서 웃길수가잇지?
6 김치 광고 받고싶어하는 기안님 큰그림
7 오기안이 돌아왔꾸나
8 기안84님 이번에 나온거는 왠지 좀 눈물이나네요
유며해지면 돈을 많이 버니 억지로라도 먹어야하고
그만 심할정도로 먹으니 왠지 기안84님과 관련이 있는거 같아 마음이 안좋네요
9 시아버지가 직접담근 김치~~
배추,젓갈 중국산  ㅋ ㅋ
쇼핑몰 문구 엄청웃김
10 미르스띤 존나 웃겨서 호흡곤란올뻔했다
11 댓글로는 까는데 아직순위권이란 얘기는 설직히우리도 다음이어떨지 기대돼자너
12 기안84  모델  PPL
13 개그 센스는 인정해줘야 된다 진짜ㅋㅋㅋㅋㅋㅋㅋ오늘 만화 미친줄ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
14 댓글알바쓰니? ㅈㄴ 나만재미없냐?
15 솔직히 재밌어요 너무 웃었어요 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 기안84님 화이팅!


댓글창 페이지 바꾸면 headers의 페이지수는 바뀌지만 전체 url은 안바뀐다. -> 댓글창은 iframe안에 있어서

In [ ]:
import pprint

In [ ]:
pprint.pprint(webtoon_dict)

In [14]:
title = input('웹툰이름')
title_id = webtoon_dict[title]['title_id']
current_no = webtoon_dict[title]['current_no']
page_no = 1     #for문 돌려야

웹툰이름신의 탑


In [15]:
title_id, current_no, page_no

('183559', '433', 1)

In [19]:
comment_url = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?'
params = {'ticket':'comic',
         'templateId' : 'webtoon',
         'pool' : 'cbox3',
         '_callback' : 'jQuery112407869252129422144_1560319769583',
         'lang' : 'ko',
         'country' : 'KR',
         'objectId' : '{}_{}'.format(title_id, current_no), #title_cuurentid no.    -> 여기에 format줘서 각 웹툰의 id와 current no.넣을거
         'pageSize' : 15,      # 댓글의 갯수를 말하는듯
         'indexSize' : 10,
         'listType' :'OBJECT',
         'pageType' : 'default',
         'page' : page_no,         #page 바꿔가면서 댓글 뽑아보자
         'refresh' : 'true',
         'sort' : 'new',
         '_' : 1560319769587}
#요청 헤더를 추가해보자
headers = {
    'Referer' : 'https://comic.naver.com/comment/comment.nhn?titleId={}&no={}'.format(title_id, current_no)  #  titleid와 current no.가 웹툼마다 달라 이걸 알려주면 어떤 웹툰에 들어갈지 파악하고 들어간다.
}                                                                                  # 전체 웹툰의 댓글 보기 위해선 웹툰의 각 화를 들어가서 읽어야한다. 그래서 위에서 titleid를 읽은것

#result = requests.get(comment_url,params = params)
result = requests.get(comment_url, headers=headers ,params = params)

result

<Response [200]>

In [22]:
import re
result_update = re.findall('769583\((.+)\);', result.text)[0]   #특수문자 인식하려면 \써줘야한다.
result_update
result_json = json.loads(result_update)
result_json
commentList = result_json['result']['commentList']
for i, comment in enumerate(commentList): #comment = dictionary
    print(i+1,comment['contents'])

1 우리집개두마리도 신수가르쳐야겠다.
2 신탑은 쿠키 안지르고는 못배겨요 진짜..
3 2900번째!!!!
4 존재뮤ㅠㅠㅠㅠㅠㅠㅠㅠㅠ
5 나왜 저 원로가 화련인거같지
6 나도 쿠키지름 ㅌㅋ
7 얼마나 쿠키를 질러대면 순위가 떨어지냐 ㅋㅋㅋ
8 역대급 몰카로 이미 삼아제가 시작된거일수도
9 자전거 체인 끊고온다
10 이 집 말장난이 맛집이네 ㅋㅋㅋㅋㅋ 하-이 코믹 클라스
11 다른웹툰들은 독자들이 순위 신경도 안쓰는데 신의탑은 2위되자마자 이렇게 화력지원되는거 보니까 역시 팬층이 두껍긴 하네요ㅋㅋ 지금도 인기있지만 2013 2014 이때당시엔 진짜 웹툰계의 신드롬이었는데
12 2위뭐냐 신의탑이 자전거 웹툰보다 낮은게 말이 되나
13 난 인간을 포기하겟다 죠죠!!!!
14 라크도 떡상하쟈!!
15 남북거북이아니냐?


여태까지는 데이터를 수집해서 변수 안에 저장
이 변수는 메모리(데이터 읽고 쓰기, 변경 가능)에 저장 (휘발성)
이를 영구적으로 저장하고 싶으면 파일 형태로 저장 (드라이브 상에 저장, 비휘발성)
-> 큰데이터를 저장, 관리하는 것이 DB
-> dataframe, pandas 보고 오자 


header 조정 안해주면 defalt값이 들어간다.

크롤링은 beautiful soup 기준으로 익히자